Settting seed and importing requirement for Metric Scripts

In [2]:
!pip install transformers datasets

import torch
seed=20
import numpy as np
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
from pathlib import Path
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset, load_metric
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments
from transformers import Trainer
import gc
import pandas as pd
import spacy
spacy = spacy.load('en_core_web_sm')
import re
from spacy.lang.en.stop_words import STOP_WORDS
def metric_fn(predictions):
    preds = predictions.predictions.argmax(axis=1)
    labels = predictions.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

    
torch.cuda.empty_cache()
gc.collect()

11

# Load DistilBert Cased

Load up the database that holds 58K answers



In [3]:
model_name='distilbert-base-cased'
model_seq_classification = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
dfBert_path='DataFrameForMetricsAnswerGen.csv'
Bert_DF=pd.read_csv(dfBert_path)  


Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bia

# Bert Model who tries to predict wheter the text is generated




In [4]:
Bert_DF=Bert_DF.dropna()
x_train,x_val,y_train,y_val=train_test_split(Bert_DF['answer'],Bert_DF['label'],test_size=0.3,random_state=20,stratify=Bert_DF['label']) # 70:30 split
train_df=pd.DataFrame(data=zip(x_train,y_train),columns=['answer','label'])
val_df=pd.DataFrame(data=zip(x_val,y_val),columns=['answer','label'])
print(train_df.describe(),val_df.describe())
train_df.to_csv('train.csv')
val_df.to_csv('val.csv')
data_files = {
    'train': str('/content/train.csv'),
    'val': str('/content/val.csv'),
}
datasets = load_dataset("csv", data_files=data_files)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_datasets = datasets.map(tokenizer, input_columns='answer', fn_kwargs={"max_length": 500, "truncation": True, "padding": "max_length"})
tokenized_datasets.set_format('torch')

for split in tokenized_datasets:
    tokenized_datasets[split] = tokenized_datasets[split].add_column('label', datasets[split]['label'])


              label
count  40786.000000
mean       0.500000
std        0.500006
min        0.000000
25%        0.000000
50%        0.500000
75%        1.000000
max        1.000000               label
count  17481.000000
mean       0.500029
std        0.500014
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000


Using custom data configuration default-3b04a2df7384673b


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3b04a2df7384673b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

  0%|          | 0/40786 [00:00<?, ?ex/s]

  0%|          | 0/17481 [00:00<?, ?ex/s]

Setting up a Trainer

In [5]:
OUT_PATH = Path("trainDir")

args = TrainingArguments(output_dir=OUT_PATH, overwrite_output_dir=True, per_device_train_batch_size=16, per_device_eval_batch_size=16, save_strategy='epoch', metric_for_best_model='eval_accuracy',load_best_model_at_end=True, greater_is_better=True, evaluation_strategy='epoch', do_train=True,
                         num_train_epochs=5, report_to='none')
trainer = Trainer(
    model=model_seq_classification,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    compute_metrics=metric_fn #using customaisble metric function
)

trainer.train()# starting to train

pred_list=trainer.predict(test_dataset=tokenized_datasets['val'])[0]
preds = pred_list.argmax(axis=1)
true_labels=tokenized_datasets['val']['label']
acc = accuracy_score(true_labels, preds)
print(true_labels,preds,acc)


The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: answer. If answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 40786
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 12750


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.662000,0.659598,0.589097,0.437642,0.693204,0.319757
2,0.631500,0.677775,0.599565,0.590978,0.603965,0.578538
3,0.535300,0.767250,0.590412,0.542959,0.614152,0.486558
4,0.437700,0.984342,0.577484,0.554093,0.586604,0.524997
5,0.351800,1.136812,0.576912,0.558817,0.583822,0.535865


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: answer. If answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 17481
  Batch size = 16
Saving model checkpoint to trainDir/checkpoint-2550
Configuration saved in trainDir/checkpoint-2550/config.json
Model weights saved in trainDir/checkpoint-2550/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: answer. If answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 17481
  Batch size = 16
Saving model checkpoint to trainDir/checkpoint-5100
Configuration saved in trainDir/checkpoint-5100/config.json
Model weights sa

tensor([0, 0, 0,  ..., 1, 0, 1]) [1 0 0 ... 0 0 1] 0.5995652422630284


,Unnamed: 0,context,answer,label
0,0,"In late 1203, John attempted to relieve Châtea...",a synchronised operation involving land-based...,1
1,1,"In late 1203, John attempted to relieve Châtea...",Château Gaillard,1
2,2,"In late 1203, John attempted to relieve Châtea...",Angevin authority in the south had been under...,1
3,3,"In late 1203, John attempted to relieve Châtea...",Eleanor died the following month,1
4,4,"In late 1203, John attempted to relieve Châtea...",August,1
...,...,...,...,...
58263,58263,Most of the space in the brain is taken up by ...,neuron cell bodies,0
58264,58264,"According to Max Clifford: Read All About It, ...",Clifford and Angela Levin,0
58265,58265,"According to the Quran, God communicated with ...",signs and revelations,0
58266,58266,Various methods have been used in the history ...,hanging,0


Making sure everything is balanced

In [7]:

Bert_DF['label'].value_counts()

1    29134
0    29133
Name: label, dtype: int64

build a Staticial Metrics to test our generated answers compared to the non-generated answers

In [8]:

genCount=0 
dataCount=0
dataCountFiltred=0
genCountFiltred=0
genWordCount=0
dataWordCount=0
totalwords=len(Bert_DF)
totalGenWords=0
totalDataWords=0
Bert_DF=Bert_DF.dropna()


for i in range(len(Bert_DF)):
   
 
  ####### Check the AVG words legnth of a generated answer versus an answer from the DataBase
  answerForLength=Bert_DF.iloc[i]['answer']
  ansCleaned =re.sub("[^0-9a-zA-Z ]", '', answerForLength)
  ansCleaned=spacy(ansCleaned)
  if Bert_DF.iloc[i]['label']==0:
        totalDataWords+=1
        dataWordCount+=len(ansCleaned)-1 #somehow one word turns out with a length of two , each word has +1 to the total
  if Bert_DF.iloc[i]['label']==1:
        totalGenWords+=1
        genWordCount+=len(ansCleaned)-1


  ans=spacy(Bert_DF.iloc[i]['answer'])
  doc=spacy(Bert_DF.iloc[i]['context'])
  docENTS=doc.ents

  myset=set()
  for item in docENTS:
    myset.add(str(item[0]))


  filtredSet =set()
  for word in docENTS:
    if (word[0]).is_stop == False:
        filtredSet.add(str(word[0]))

  ###### Count how many part of answers are included in the main entities of the text. for each kind of answer, generated or one from the Database
  for token in ans:
    tokens =re.sub("[^0-9a-zA-Z]", '', token.text)
    #print(tokens)
    if tokens in myset:
      if Bert_DF.iloc[i]['label']==0:
        dataCount+=1
      if Bert_DF.iloc[i]['label']==1:
        genCount+=1
  ###### Count how many part of answers are included in the main entities of the text. for each kind of answer, generated or one from the Database , making sure to filter stop words before counting.

    if tokens in filtredSet:
      if Bert_DF.iloc[i]['label']==0:
        dataCountFiltred+=1
      if Bert_DF.iloc[i]['label']==1:
        genCountFiltred+=1
      
        
print(f'Data Count: {dataCount}, Gen Count: {genCount}')    
print(f'Data CountFiltred: {dataCountFiltred}, Gen CountFiltred: {genCountFiltred}')    

print('Printing AVG length below:')
print(f'totalDataWords: {totalDataWords},  dataWordCount: {dataWordCount} AVG words per answer Data: {dataWordCount/totalDataWords}')
print(f'totalGenWords: {totalGenWords},  genWordCount: {genWordCount}, AVG words per answer Gen: {genWordCount/totalGenWords}')
print(f'total words :{totalwords}')



Data Count: 21888, Gen Count: 25467
Data CountFiltred: 17047, Gen CountFiltred: 18349
Printing AVG length below:
totalDataWords: 29133,  dataWordCount: 63913 AVG words per answer Data: 2.1938351697387843
totalGenWords: 29134,  genWordCount: 143339, AVG words per answer Gen: 4.919990389235944
total words :58267
